In [107]:
import sqlite3
import csv
import pandas as pd
import os
import re

import math
from tqdm import tqdm
import multiprocessing as mp
import time
import sched
import ast
import numpy as np
import os

folder_prefix = '/home/jovyan/work/'


In [108]:
#was originally charters_full_2015_15_250.pkl for xxl vm , but is charters_full_2015_250_new.pkl in large vm for testing purposes
full_250_df = pd.read_pickle(folder_prefix + "nowdata/charters_full_2015_250_new.pkl")


In [50]:
full_250_df

,CMO_NAME,CMO_MEMSUM,SCH_NAME,CMO_STATE,CMO_SCHNUM,CMO_URL,CMO_NUMSTATES,CMO_ALLSTATES,CMO_SECTOR,CMO_NUMSTUDENTS_CREDO17,...,YEAR_OPENED,YEAR_CLOSED,SEARCH16,TOTETH,PCTETH,AGE,PCTFRL,PLACE,STR,WEBTEXT_METHOD
0,AAEC,1320.0,ARIZONA AGRIBUSINESS & EQUINE CENTER - ESTRELLA,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,NaN,NaN,...,2010.0,NaN,ARIZONA AGRIBUSINESS & EQUINE CENTER - ESTRELL...,239.0,0.497917,5.0,-0.002083,Suburb,-480.000000,0
1,Arizona Agribusiness & Equine Center,1320.0,ARIZONA AGRIBUSINESS & EQUINE CENTER - ESTRELLA,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,CMO,1220.0,...,2010.0,NaN,ARIZONA AGRIBUSINESS & EQUINE CENTER - ESTRELL...,239.0,0.497917,5.0,-0.002083,Suburb,-480.000000,0
2,AAEC,1320.0,AAEC - SMCC CAMPUS,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,NaN,NaN,...,2005.0,NaN,"AAEC - SMCC CAMPUS 7050 S. 24th Street , Phoe...",396.0,0.912442,10.0,0.649770,City,-434.000000,0
3,Arizona Agribusiness & Equine Center,1320.0,AAEC - SMCC CAMPUS,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,CMO,1220.0,...,2005.0,NaN,"AAEC - SMCC CAMPUS 7050 S. 24th Street , Phoe...",396.0,0.912442,10.0,0.649770,City,-434.000000,0
4,AAEC,1320.0,AAEC - PARADISE VALLEY,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,NaN,NaN,...,1998.0,NaN,AAEC - PARADISE VALLEY 17811 North 32nd Stree...,81.0,0.199507,17.0,-0.002463,City,-406.000000,0
5,Arizona Agribusiness & Equine Center,1320.0,AAEC - PARADISE VALLEY,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,CMO,1220.0,...,1998.0,NaN,AAEC - PARADISE VALLEY 17811 North 32nd Stree...,81.0,0.199507,17.0,-0.002463,City,-406.000000,0
6,ACADEMY OF SKILLS AND KNOWLEDGE,1161.0,CUMBERLAND ACADEMY MIDDLE,TX,4.0,http://www.cumberlandacademy.com/,1.0,TX,NaN,NaN,...,2013.0,NaN,"CUMBERLAND ACADEMY MIDDLE 1040 SHILOH RD , TY...",240.0,0.471513,2.0,0.379175,City,12.469378,0
7,ACADEMY OF SKILLS AND KNOWLEDGE,1161.0,CUMBERLAND H S,TX,4.0,http://www.cumberlandacademy.com/,1.0,TX,NaN,NaN,...,2015.0,NaN,"CUMBERLAND H S 7200 PALUXY RD , TYLER TX 7570...",59.0,0.385621,0.0,NaN,Rural,NaN,0
8,ACADEMY OF SKILLS AND KNOWLEDGE,1161.0,CUMBERLAND ACADEMY,TX,4.0,http://www.cumberlandacademy.com/,1.0,TX,NaN,NaN,...,1998.0,NaN,"CUMBERLAND ACADEMY 1340 SHILOH RD , TYLER TX ...",217.0,0.434870,17.0,0.408818,City,13.773116,0
9,ACCELERATED INTERMEDIATE ACADEMY,281.0,ACCELERATED INTERDISCIPLINARY ACAD,TX,3.0,http://www.aiacharterschools.org/,1.0,TX,NaN,NaN,...,2004.0,NaN,ACCELERATED INTERDISCIPLINARY ACAD 12825 SUMM...,276.0,0.982206,11.0,0.864769,City,16.676558,0


In [179]:
parsed_df_6 = pd.read_csv(folder_prefix + "nowdata/parsing/parsed_df_6.csv", sep="\t", low_memory=False, encoding="utf-8")

In [52]:
parsed_df_5 = pd.read_csv(folder_prefix + "nowdata/parsing/parsed_df_5.csv", sep="\t", low_memory=False, encoding="utf-8")

In [53]:
parsed_df_4 = pd.read_csv(folder_prefix + "nowdata/parsing/parsed_df_4.csv", sep="\t", low_memory=False, encoding="utf-8")

In [54]:
parsed_df_3 = pd.read_csv(folder_prefix + "nowdata/parsing/parsed_df_3.csv", sep="\t", low_memory=False, encoding="utf-8")

In [55]:
parsed_df_2 = pd.read_csv(folder_prefix + "nowdata/parsing/parsed_df_2.csv", sep="\t", low_memory=False, encoding="utf-8")

In [184]:
parsed_df_1 = pd.read_csv(folder_prefix + 'nowdata/parsing/parsed_df_1.tar.gz', compression='gzip', sep='\t', low_memory=False, encoding="utf-8")


In [57]:
lookup = pd.DataFrame(columns=['NCESSCH','OVERLAPS_REMOVED','DATA_SOURCE'])

In [58]:
for i in parsed_df_1.columns:
    
    print(i)
    



parsed_df_1.csv
CMO_MEMSUM
SCH_NAME
CMO_STATE
CMO_SCHNUM
CMO_URL
CMO_NUMSTATES
CMO_ALLSTATES
CMO_SECTOR
CMO_NUMSTUDENTS_CREDO17
CMO_TYPE
CMO_WEBTEXT
SURVYEAR
FIPST
STABR
SEANAME
LEAID
ST_LEAID
SCHID
ST_SCHID
NCESSCH
MSTREET1
MSTREET2
MSTREET3
MCITY
MSTATE
MZIP
MZIP4
PHONE
LSTREET1
LSTREET2
LSTREET3
LCITY
LSTATE
LZIP
LZIP4
UNION
OUT_OF_STATE_FLAG
SCH_TYPE_TEXT
SCH_TYPE
RECON_STATUS
GSLO
GSHI
LEVEL
VIRTUAL
BIES
SY_STATUS_TEXT
SY_STATUS
UPDATED_STATUS_TEXT
UPDATED_STATUS
EFFECTIVE_DATE
CHARTER_TEXT
G13OFFERED
AEOFFERED
UGOFFERED
NOGRADES
CHARTAUTH1
CHARTAUTHN1
CHARTAUTH2
CHARTAUTHN2
IGOFFERED
WEBSITE
FRELCH
REDLCH
AE
TOTAL
AM
AMALM
AMALF
AS
ASALM
ASALF
HI
HIALM
HIALF
BL
BLALM
BLALF
WH
WHALM
WHALF
HP
HPALM
HPALF
TR
TRALM
TRALF
TITLEI_TEXT
TITLEI_STATUS
STITLEI
SHARED_TIME
MAGNET_TEXT
NSLPSTATUS_TEXT
NSLPSTATUS_CODE
NAME
OPSTFIPS
LSTREE
STFIP15
CNTY15
NMCNTY15
LOCALE15
LAT1516
LON1516
CBSA15
NMCBSA15
CBSATYPE15
CSA15
NMCSA15
NECTA15
NMNECTA15
CD15
SLDL15
SLDU15
LSTREET114
LSTREET214
LSTREET

In [59]:
parsed_df_6['NCESSCH'].isna().sum()

0

In [60]:
df_6_ids = list(set(parsed_df_6['NCESSCH']))
df_6_ids

[480023709189.0,
 120129004043.0,
 120129004044.0,
 120108007441.0,
 120132007956.0,
 40061403158.0,
 290060103194.0,
 50007401501.0,
 120132003875.0,
 120123003429.0,
 120108007463.0,
 69103513640.0,
 220117000234.0,
 120108008490.0,
 120108007470.0,
 480002508343.0,
 120108004926.0,
 63441012800.0,
 120108004929.0,
 50005501508.0,
 120108004933.0,
 40061302861.0,
 120105003597.0,
 480027213391.0,
 61495011927.0,
 80426001505.0,
 50005500513.0,
 480023712866.0,
 120126008427.0,
 480023712366.0,
 80002002033.0,
 480023712372.0,
 120108003956.0,
 61495010422.0,
 120108008062.0,
 63023010431.0,
 450390101639.0,
 110007200405.0,
 120126004387.0,
 370041803428.0,
 120159001775.0,
 220117000880.0,
 50006100658.0,
 480026613428.0,
 110006000310.0,
 480007111352.0,
 120126007489.0,
 110005300428.0,
 120123008208.0,
 290059903197.0,
 220117000926.0,
 40015701728.0,
 120105001702.0,
 110005300455.0,
 40024402153.0,
 480023711465.0,
 69103511792.0,
 470318002423.0,
 110005100301.0,
 120132007698

In [61]:
c=0
for i in df_6_ids:
    if i in list(so_far):
        c+=1
    
c

NameError: name 'so_far' is not defined

In [62]:
so_far = parsed_df_1['NCESSCH'].append(parsed_df_2['NCESSCH']).append(parsed_df_3['NCESSCH']).append(parsed_df_4['NCESSCH']).append(parsed_df_5['NCESSCH']).append(parsed_df_6['NCESSCH'])

In [63]:
so_far_unique = list(set(so_far))

In [64]:
len(so_far_unique)

6136

In [65]:
len(so_far)

6579

In [66]:
cleaned_list = [x for x in so_far_unique if str(x) != 'nan']


In [67]:
len(cleaned_list) # true unique ids that have been seen already

6135

In [68]:
parsed_df_1.shape[0]+parsed_df_2.shape[0]+parsed_df_3.shape[0]+parsed_df_4.shape[0]+parsed_df_5.shape[0]+parsed_df_6.shape[0]

6579

In [69]:
lookup['NCESSCH'] = full_250_df['NCESSCH']

In [70]:
count = 0
minus = 0
u = set(full_250_df['NCESSCH'])
for nc in u:
    indices = lookup.index[lookup['NCESSCH'] == nc].tolist()
    if nc in cleaned_list: #if already parsed 
        for i in indices:
            lookup.loc[i, 'OVERLAPS_REMOVED'] = 1
            if nc in df_6_ids:
                lookup.loc[i, 'DATA_SOURCE'] = 'full_15_250'
            else :
                lookup.loc[i, 'DATA_SOURCE'] = 'full'
        count+=1
    else:
        
        for i in indices:
            lookup.loc[i, 'OVERLAPS_REMOVED'] = 0
            lookup.loc[i, 'DATA_SOURCE'] = 'full_15_250'
        minus+=1
        
        
        

In [71]:
count

6135

In [72]:
minus

4830

In [73]:
count+minus

10965

In [74]:
six = lookup[(lookup['DATA_SOURCE'] == 'full_15_250') & (lookup['OVERLAPS_REMOVED'] == 1)]

In [75]:
len(set(six['NCESSCH']))

99

In [76]:
lookup.shape[0]

11521

In [77]:
lookup[lookup['OVERLAPS_REMOVED'] == 1].shape[0] #this number is bigger than len(cleaned_list) cz lookup has repeated rows
#count is 6036 which means it made 1, 6036 times -> which is the leangth of cleaned list


6347

In [78]:
len(so_far)

6579

In [79]:
lookup.isna().sum()

NCESSCH             0
OVERLAPS_REMOVED    0
DATA_SOURCE         0
dtype: int64

In [80]:
len(set(so_far))

6136

In [81]:
lookup.to_csv(folder_prefix + "/nowdata/parsing/lookup.csv", mode="w", index=False, header=lookup.columns.values, sep="\t", encoding="utf-8")

In [82]:
s = pd.read_csv(folder_prefix + "/nowdata/parsing/lookup.csv", sep="\t", low_memory=False, encoding="utf-8")

In [83]:
s

,NCESSCH,OVERLAPS_REMOVED,DATA_SOURCE
0,4.008280e+10,0,full_15_250
1,4.008280e+10,0,full_15_250
2,4.004170e+10,0,full_15_250
3,4.004170e+10,0,full_15_250
4,4.001060e+10,0,full_15_250
5,4.001060e+10,0,full_15_250
6,4.800034e+11,1,full
7,4.800034e+11,1,full
8,4.800034e+11,1,full
9,4.800192e+11,1,full


In [85]:
s[s['OVERLAPS_REMOVED'] == 1].shape[0]

6347

In [86]:
s.isna().sum()

NCESSCH             0
OVERLAPS_REMOVED    0
DATA_SOURCE         0
dtype: int64

In [87]:
unseen = s[s['OVERLAPS_REMOVED'] == 0]
unseen

,NCESSCH,OVERLAPS_REMOVED,DATA_SOURCE
0,4.008280e+10,0,full_15_250
1,4.008280e+10,0,full_15_250
2,4.004170e+10,0,full_15_250
3,4.004170e+10,0,full_15_250
4,4.001060e+10,0,full_15_250
5,4.001060e+10,0,full_15_250
27,4.200839e+11,0,full_15_250
28,4.200043e+11,0,full_15_250
29,1.200180e+11,0,full_15_250
30,1.200180e+11,0,full_15_250


In [88]:
5279 + 6242



11521

In [89]:
li = list(unseen['NCESSCH'])

In [90]:
len(li)

5174

In [91]:
p = full_250_df[full_250_df['NCESSCH'].isin(li)]

In [154]:
type(full_250_df['WEBTEXT'][0])

list

In [162]:
new_data = pd.DataFrame(columns=['id', 'name'])
new_data['id'] = pd.Series([[100], [109], [40], [32], [16], [19], [8], [33], [20], [21], [54], [3]])
new_data['name'] = pd.Series(["Billy", "Krutika", "Joe", "fdjs", "gfd", "re","fde", "t", "a", "e", "bf", "tr"])
# i = new_data.head(2)
# new_data = new_data.append(i)
new_data

,id,name
0,[100],Billy
1,[109],Krutika
2,[40],Joe
3,[32],fdjs
4,[16],gfd
5,[19],re
6,[8],fde
7,[33],t
8,[20],a
9,[21],e


In [163]:
i = new_data.head()

In [164]:
n = pd.concat([new_data, i])

In [165]:
n

,id,name
0,[100],Billy
1,[109],Krutika
2,[40],Joe
3,[32],fdjs
4,[16],gfd
5,[19],re
6,[8],fde
7,[33],t
8,[20],a
9,[21],e


In [147]:
def add_val(li):
    print(li)
    k = [x + 1 for x in li]
    k.append([90])
    return k

In [148]:
def parse_df(old_list):
    new_list = add_val(old_list)
    return new_list

In [149]:
new_data['id']

0     [100]
1     [109]
2      [40]
3      [32]
4      [16]
5      [19]
6       [8]
7      [33]
8      [20]
9      [21]
10     [54]
11      [3]
Name: id, dtype: object

In [150]:
new_data['id'] = new_data['id'].apply(parse_df)
new_data

[100]
[109]
[40]
[32]
[16]
[19]
[8]
[33]
[20]
[21]
[54]
[3]


,id,name
0,"[101, [90]]",Billy
1,"[110, [90]]",Krutika
2,"[41, [90]]",Joe
3,"[33, [90]]",fdjs
4,"[17, [90]]",gfd
5,"[20, [90]]",re
6,"[9, [90]]",fde
7,"[34, [90]]",t
8,"[21, [90]]",a
9,"[22, [90]]",e


In [111]:
arr_of_dfs = np.array_split(new_data, len(new_data['id']))


In [112]:
[ x + 1 for x in [0,1] ]


[1, 2]

In [153]:
def chunk_assign(df_chunk): #Jaren chunk by chunk 
    global num
    #global temp_df
    global temp_df
    
    #already_cleaned_chunk = df_chunk.loc[df_chunk['OVERLAPS_REMOVED'] == 1] don't do anything with this
    
    df_chunk['id'] = df_chunk['id'].apply(parse_df)
    
    temp_df = temp_df.append(df_chunk, ignore_index = True)
    print(temp_df['id'].values)

    
    
            
    if num == 0: # Save first slice to new file (overwriting if needed)
        temp_df.to_csv(folder_prefix + "nowdata/parsing/temp_test.csv", mode="w", index=False, header=df_chunk.columns.values, sep="\t", encoding="utf-8")
        
    elif (num > 0) and (num % 10 == 0):
        temp_df.to_csv(folder_prefix + "nowdata/parsing/temp_test.csv", mode="a", index=False, header=False, sep="\t", encoding="utf-8")


    num+=1
    
    return df_chunk

In [156]:
num = 0
temp_df = pd.DataFrame(columns=new_data.columns)
numcpus = len(os.sched_getaffinity(0)) # Detect and assign number of available CPUs
p = mp.Pool(numcpus)
c = 0
j = p.map(chunk_assign, arr_of_dfs)

hi
hi
hi
hi
hi
[list([41])]
[list([33])]
hi
hi
[list([9])]
[list([17])]
hi
hi
[list([101])]
hi
hi
hi
[list([34])]
[list([110])]
[list([21])]
[list([22])]
[list([20])]
[list([55])]
[list([4])]


In [157]:
j

[      id   name
 0  [101]  Billy,       id     name
 1  [110]  Krutika,      id name
 2  [41]  Joe,      id  name
 3  [33]  fdjs,      id name
 4  [17]  gfd,      id name
 5  [20]   re,     id name
 6  [9]  fde,      id name
 7  [34]    t,      id name
 8  [21]    a,      id name
 9  [22]    e,       id name
 10  [55]   bf,      id name
 11  [4]   tr]

In [125]:
arr_of_dfs

[      id   name
 0  [100]  Billy,       id     name
 1  [109]  Krutika,      id name
 2  [40]  Joe,      id  name
 3  [32]  fdjs,      id name
 4  [16]  gfd,      id name
 5  [19]   re,     id name
 6  [8]  fde,      id name
 7  [33]    t,      id name
 8  [20]    a,      id name
 9  [21]    e,       id name
 10  [54]   bf,      id name
 11  [3]   tr]

In [149]:
t = pd.read_csv(folder_prefix + "nowdata/parsing/temp_test.csv", sep="\t", low_memory=False, encoding="utf-8")

In [150]:
t

,id,name
0,[55],bf


In [7]:
full_250_df = pd.read_csv(folder_prefix + "nowdata/parsing/parsed_df_2.csv", sep="\t", low_memory=False, encoding="utf-8")


In [94]:
arr_of_dfs = np.array_split(full_250_df, len(full_250_df['WEBTEXT']))

In [115]:
type(arr_of_dfs[0])

pandas.core.frame.DataFrame

In [ ]:
def add_1()

In [ ]:
k= full_250_df.head(5)

In [95]:
full_250_df.shape[0]

11521

In [96]:
len(arr_of_dfs)

11521

In [113]:
type(arr_of_dfs)

list

In [98]:
k = [1,3]
k.append([6,9])
k


[1, 3, [6, 9]]

In [99]:
ten_count = int(round(len(arr_of_dfs) /10))
ten_count

1152

In [100]:
indices_arr = np.arange(len(arr_of_dfs))
ind_sublists = np.array_split(indices_arr, ten_count)


In [112]:
type(ind_sublists)

list

In [102]:
for i in ind_sublists :
    print(type(i))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.nd

In [103]:
len(arr_of_dfs)

11521

In [104]:
ind_sublists[1][0]

11

In [105]:
ind_sublists[1][len(ind_sublists[1]) - 1]

20

In [106]:
len(arr_of_dfs[ind_sublists[0][0]: ind_sublists[0][len(ind_sublists[0]) - 1] + 1])

11

In [107]:
arr_of_dfs[ind_sublists[1][0]: ind_sublists[1][len(ind_sublists[1]) -1] +1]

[                            CMO_NAME  CMO_MEMSUM          SCH_NAME CMO_STATE  \
 11  ACCELERATED INTERMEDIATE ACADEMY       281.0  AIA LANCASTER EL        TX   
 
     CMO_SCHNUM                            CMO_URL  CMO_NUMSTATES  \
 11         3.0  http://www.aiacharterschools.org/            1.0   
 
    CMO_ALLSTATES CMO_SECTOR  CMO_NUMSTUDENTS_CREDO17      ...        \
 11            TX        NaN                      NaN      ...         
 
    YEAR_OPENED YEAR_CLOSED                                           SEARCH16  \
 11      2010.0         NaN  AIA LANCASTER EL  901 E BELTLINE RD , LANCASTE...   
 
     TOTETH PCTETH  AGE  PCTFRL  PLACE  STR WEBTEXT_METHOD  
 11     NaN    NaN  5.0     NaN  Rural  NaN              0  
 
 [1 rows x 392 columns],
                             CMO_NAME  CMO_MEMSUM              SCH_NAME  \
 12  ACCELERATED INTERMEDIATE ACADEMY       281.0  AIA LANCASTER MIDDLE   
 
    CMO_STATE  CMO_SCHNUM                            CMO_URL  CMO_NUMSTATES  \
 12 

In [108]:
for sub_array in ind_sublists:
    arr_dfs_chunk = arr_of_dfs[sub_array[0]: sub_array[len(sub_array) -1] +1]
    print(str(sub_array[0]) + " : " + str(sub_array[len(sub_array) -1]))

0 : 10
11 : 20
21 : 30
31 : 40
41 : 50
51 : 60
61 : 70
71 : 80
81 : 90
91 : 100
101 : 110
111 : 120
121 : 130
131 : 140
141 : 150
151 : 160
161 : 170
171 : 180
181 : 190
191 : 200
201 : 210
211 : 220
221 : 230
231 : 240
241 : 250
251 : 260
261 : 270
271 : 280
281 : 290
291 : 300
301 : 310
311 : 320
321 : 330
331 : 340
341 : 350
351 : 360
361 : 370
371 : 380
381 : 390
391 : 400
401 : 410
411 : 420
421 : 430
431 : 440
441 : 450
451 : 460
461 : 470
471 : 480
481 : 490
491 : 500
501 : 510
511 : 520
521 : 530
531 : 540
541 : 550
551 : 560
561 : 570
571 : 580
581 : 590
591 : 600
601 : 610
611 : 620
621 : 630
631 : 640
641 : 650
651 : 660
661 : 670
671 : 680
681 : 690
691 : 700
701 : 710
711 : 720
721 : 730
731 : 740
741 : 750
751 : 760
761 : 770
771 : 780
781 : 790
791 : 800
801 : 810
811 : 820
821 : 830
831 : 840
841 : 850
851 : 860
861 : 870
871 : 880
881 : 890
891 : 900
901 : 910
911 : 920
921 : 930
931 : 940
941 : 950
951 : 960
961 : 970
971 : 980
981 : 990
991 : 1000
1001 : 1010
1011 : 

In [109]:
ind_sublists

[array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
 array([11, 12, 13, 14, 15, 16, 17, 18, 19, 20]),
 array([21, 22, 23, 24, 25, 26, 27, 28, 29, 30]),
 array([31, 32, 33, 34, 35, 36, 37, 38, 39, 40]),
 array([41, 42, 43, 44, 45, 46, 47, 48, 49, 50]),
 array([51, 52, 53, 54, 55, 56, 57, 58, 59, 60]),
 array([61, 62, 63, 64, 65, 66, 67, 68, 69, 70]),
 array([71, 72, 73, 74, 75, 76, 77, 78, 79, 80]),
 array([81, 82, 83, 84, 85, 86, 87, 88, 89, 90]),
 array([ 91,  92,  93,  94,  95,  96,  97,  98,  99, 100]),
 array([101, 102, 103, 104, 105, 106, 107, 108, 109, 110]),
 array([111, 112, 113, 114, 115, 116, 117, 118, 119, 120]),
 array([121, 122, 123, 124, 125, 126, 127, 128, 129, 130]),
 array([131, 132, 133, 134, 135, 136, 137, 138, 139, 140]),
 array([141, 142, 143, 144, 145, 146, 147, 148, 149, 150]),
 array([151, 152, 153, 154, 155, 156, 157, 158, 159, 160]),
 array([161, 162, 163, 164, 165, 166, 167, 168, 169, 170]),
 array([171, 172, 173, 174, 175, 176, 177, 178, 179, 180]),
 array

In [110]:
np.array_split(indices_arr, ten_count) = [1222,7753,4,543543,2323]


SyntaxError: can't assign to function call (<ipython-input-110-963d4029e3d2>, line 1)

In [111]:
np.arange(len(arr_of_dfs))

array([    0,     1,     2, ..., 11518, 11519, 11520])

In [109]:
parsed_df_7 = pd.read_csv(folder_prefix + "nowdata/parsing/parsed_df_7.csv", sep="\t", low_memory=False, encoding="utf-8")

In [112]:
parsed_df_7['WEBTEXT']= parsed_df_7['WEBTEXT'].apply(ast.literal_eval)

In [113]:
parsed_df_7.shape[0]

344

In [114]:
parsed_df_7['WEBTEXT'][5]

[('https://www.aaechighschools.com/index.php',
  'False',
  '0',
  "602.297.8500\nPrograms\nAcademics\nEquine Studies\nMath & Sciences\nVeterinary & Medical\nOur Schools\nSouth Mountain\nCalendar\nParadise Valley\nCalendar\nRed Mountain\nCalendar\nEstrella Mountain\nCalendar\nPrescott Valley\nCalendar\nMesa\nCalendar\nFor Parents\nStudent Life\nAbout AAEC\nHistory & Philosophy\nGoverning Board\nCommunity Relations\nRegistration\nOther Scholarship Opportunities\nResources\nAccreditation\nSPED\nCareers\nAAEC – In the News\nResource Guides\nPolicies\nFAQ\nBlog\nCommunity Service Events\nAnnual Financial Reports\nPre-Enroll\nContact Us\nBlog\nMesa Campus Now Open!\nAAEC is also building a brand new facility next to MCC in Mesa!!! Call today for details on how you can benefit from AAEC 480-222-3999.\nJoin Today\nSpring semester college classes have started for our High School students. Call today for details on how you can benefit from AAEC.\nPrograms\nStudents who enroll in our college pre

In [57]:
parsed_df_7['WEBTEXT'][50]

[('https://www.somersetdade.com/', 'False', '0', '')]

In [58]:
id = parsed_df_7['NCESSCH'][50]
id

120039003974.0

In [71]:
full_250_df['WEBTEXT'][0][0][3]

"Home\n | \nParent / Student Login \nCall Today!\n602.297.8500\nPrograms\nAcademics\nEquine Studies\nMath & Sciences\nVeterinary & Medical\nOur Schools\nSouth Mountain\nCalendar\nParadise Valley\nCalendar\nRed Mountain\nCalendar\nEstrella Mountain\nCalendar\nPrescott Valley\nCalendar\nMesa\nCalendar\nFor Parents\nStudent Life\nAbout AAEC\nHistory & Philosophy\nGoverning Board\nCommunity Relations\nRegistration\nOther Scholarship Opportunities\nResources\nAccreditation\nSPED\nCareers\nAAEC – In the News\nResource Guides\nPolicies\nFAQ\nBlog\nCommunity Service Events\nAnnual Financial Reports\nPre-Enroll\nContact Us\nBlog\nEstrella Mountain\nAAEC's Estrella Mountain public charter school is an early college high school offering students the opportunity to earn college credits.\nEstrella Mountain\nSafer campuses with small class sizes.\nEstrella Mountain\nLearn from highly qualified teachers. \nEstrella Mountain\nLet your interests guide your education.\nEstrella Mountain\nBe a part of St

In [98]:
# good_ids = []
# for i in parsed_df_7['NCESSCH']:
#     e = full_250_df[full_250_df['NCESSCH'] == i ]
#     s = parsed_df_7[parsed_df_7['NCESSCH'] == i ]
#     for li_tup in s['WEBTEXT']:
#         for tup in li_tup:
#             if len(li_tup) ==1 and len(e['WEBTEXT'].iloc[0]) == 1 and len(tup) >3 and len(e['WEBTEXT'].iloc[0][0]) >3:
#                 if len(tup[3]) <= 1 and len(e['WEBTEXT'].iloc[0][0][3]) >1:
#                     ids.append(i)
#                     print(e['WEBTEXT'].iloc[0][0][3])


404 Not Found. We can not find the page you are looking for.
404 Not Found. We can not find the page you are looking for.
404 Not Found. We can not find the page you are looking for.
404 Not Found. We can not find the page you are looking for.
404 Not Found. We can not find the page you are looking for.
404 Not Found. We can not find the page you are looking for.


In [115]:
bad_ids = []
for i in parsed_df_7['NCESSCH']:
    e = full_250_df[full_250_df['NCESSCH'] == i ].head(1)
    s = parsed_df_7[parsed_df_7['NCESSCH'] == i ].head(1)
    for li_tup in s['WEBTEXT']:
        if len(li_tup) != len(e['WEBTEXT'].iloc[0]):
            bad_ids.append(i)
            print(i)
      

In [116]:
len(bad_ids)
bad_ids

[]

In [125]:
less = []
for i in parsed_df_7['NCESSCH']:
    e = full_250_df[full_250_df['NCESSCH'] == i ].head(1)
    s = parsed_df_7[parsed_df_7['NCESSCH'] == i ].head(1)
    for li_tup in s['WEBTEXT']:
        if len(li_tup) >=1 and len(li_tup[0][3]) <=1:
            less.append(i)
            print(i)

120018008546.0
120018008546.0
120132007960.0
120132007960.0
120018004054.0
120018004054.0
120048007908.0
120048007908.0
120018004471.0
120018004471.0
120039007428.0
120039007428.0
120039007554.0
120039007554.0
120039007554.0
120039008160.0
120039008160.0
120039008153.0
120039008153.0
120150008220.0
120150008220.0
120150008275.0
120150008275.0
120150008251.0
120150008251.0
120150008231.0
120150008231.0
120018003823.0
120018003823.0
120150008535.0
120150008535.0
120018004472.0
120018004472.0
120039003974.0
120039003974.0
120039003974.0
120018004463.0
120018004463.0
120039007525.0
120039007525.0
120039007851.0
120039007851.0
120039007851.0
120018007893.0
120018007893.0
120018007893.0
120039007884.0
120039007884.0
120039007884.0
120018007832.0
120018007832.0
120018007832.0
120018007912.0
120018007912.0
120018007912.0
120039004071.0
120039004071.0
120039004071.0
120039004071.0
320000100830.0
320000100830.0
320000100830.0
320000100830.0
120039003699.0
120039003699.0
120039003558.0
1200390035

In [127]:
len(less)

172

In [130]:
not_same = []
for i in less:
    e = full_250_df[full_250_df['NCESSCH'] == i ].head(1)
    s = parsed_df_7[parsed_df_7['NCESSCH'] == i ].head(1)
    for li_tup in e['WEBTEXT']:
        if not(len(li_tup) >=1 and len(li_tup[0][3]) <=1):#we already know len(li_tup) >=1 and len(li_tup[0][3]) <=1
            not_same.append(i)
            print(i)

62271012077.0
62805008440.0
63255013891.0
63801011423.0
63867012365.0
62223008323.0


In [136]:
full_250_df[full_250_df['NCESSCH'] == 62223008323.0]['WEBTEXT'].iloc[0] 
# all are just reapeated pages of "404 Not Found. We can not find the page you are looking for."


[('https://aspirepublicschools.org/schools/regions/california-schools/aspire-vincent-shalvey-academy',
  'False',
  '0',
  '404 Not Found. We can not find the page you are looking for.')]

In [140]:
parsed_df_7[parsed_df_7['NCESSCH'] == 62223008323.0]['WEBTEXT'].iloc[0] 
#gets rid of repeats all are just reapeated pages of "404 Not Found. We can not find the page you are looking for."


[('https://aspirepublicschools.org/schools/regions/california-schools/aspire-vincent-shalvey-academy',
  'False',
  '0',
  '.')]

In [143]:
parsed_df_7['WEBTEXT'].iloc[30]


[('http://www.somersetacademybay.com/', 'False', '0', '')]

In [148]:
i_d = parsed_df_7['NCESSCH'].iloc[30]
id

120039003974.0

In [149]:
full_250_df[full_250_df['NCESSCH'] == i_d ]['WEBTEXT'].iloc[0]

[('http://www.somersetacademybay.com/', 'False', '0', '')]

In [160]:
#how many values empty in parsed_df
empty_ids = []

for nc in parsed_df_7['NCESSCH']:
    nid = parsed_df_7[parsed_df_7['NCESSCH']== nc].index.values.astype(int)[0]
    i = parsed_df_7['WEBTEXT'].loc[nid]
    if len(i) ==1 and len(i[0][3]) <=1:
        empty_ids.append(nc)
    
    


In [165]:
len(set(empty_ids))

96

In [162]:
print(type(empty_ids))

<class 'list'>


In [163]:
full_empty = full_250_df[full_250_df['NCESSCH'].isin(empty_ids)]

In [166]:
unique = set(full_empty['NCESSCH'])
print(len(unique))

96


In [120]:
full_250_df[full_250_df['NCESSCH'] == 40010601892.0 ]['WEBTEXT'][0]

[('https://www.aaechighschools.com/index.php',
  'False',
  '0',
  "Home\n | \nParent / Student Login \nCall Today!\n602.297.8500\nPrograms\nAcademics\nEquine Studies\nMath & Sciences\nVeterinary & Medical\nOur Schools\nSouth Mountain\nCalendar\nParadise Valley\nCalendar\nRed Mountain\nCalendar\nEstrella Mountain\nCalendar\nPrescott Valley\nCalendar\nMesa\nCalendar\nFor Parents\nStudent Life\nAbout AAEC\nHistory & Philosophy\nGoverning Board\nCommunity Relations\nRegistration\nOther Scholarship Opportunities\nResources\nAccreditation\nSPED\nCareers\nAAEC – In the News\nResource Guides\nPolicies\nFAQ\nBlog\nCommunity Service Events\nAnnual Financial Reports\nPre-Enroll\nContact Us\nBlog\nMesa Campus Now Open!\nAAEC is also building a brand new facility next to MCC in Mesa!!! Call today for details on how you can benefit from AAEC 480-222-3999.\nJoin Today\nSpring semester college classes have started for our High School students. Call today for details on how you can benefit from AAEC.\

In [93]:
full_250_df[full_250_df['NCESSCH'] == 120018007837.0 ]['WEBTEXT'][0]

[('http://www.somersetwm.com/wordpress/?page_id=1893',
  'False',
  '1',
  ' Phone: (954) 390-0971\r\t\t\t\t\r\t\t\t\t\t\t\t\t\t\namarucci@somersetwm.com\nContact\nNews\nPartners\nRegister\nHome\nAbout Us\nAccreditation\nAcademic Performance\nContact\nPrincipal’s Message\nParent Liaison\nEducation Service Provider\nFAQ\nGovernance & Financials\nBoard Members\nBoard Minutes\nBoard Meetings\nBoard Speaker Form\nBudgets and Financials\nNews and Bulletin\nSchool Mission\nSomerset Academy Inc.\nTitle 1\nPartners\nPublic Records Request\nWhat are Charter Schools?\nWho We Are\nRegister\nStudents / Parents\nKey Information Bulletin\nSAC/Parent Meetings\nTutoring\nUniform Policy\nStudent Links\nClubs\nChild Abuse and Neglect Policy\nStudent Code of Conduct\nStudent Progression Plan\nSupply Lists\nSummer Reading Lists\nVolunteer Opportunities\nCalendars\nActivities Calendar\nBroward County Calendar 2017-2018\nCurriculum\nElementary Curriculum\nMiddle School Curriculum Guide\nLunch Menu\nAthletic

In [97]:
u = parsed_df_7[parsed_df_7['NCESSCH'] == 120018007837.0 ]['WEBTEXT']
count = 0
for row in u:
    print(len(row))


42


In [89]:
count

1

In [96]:
#index on kept in parsed_df_7
len(parsed_df_7[parsed_df_7['NCESSCH'] == 120018007837.0 ]['WEBTEXT'].iloc[0])

42

In [92]:
type(parsed_df_7['WEBTEXT'][30])

list

In [77]:
for i in ids:
    e = full_250_df[full_250_df['NCESSCH'] == i ]
    print(e['WEBTEXT'][0])

[('http://www.somersetwm.com/wordpress/?page_id=1893', 'False', '1', ' Phone: (954) 390-0971\r\t\t\t\t\r\t\t\t\t\t\t\t\t\t\namarucci@somersetwm.com\nContact\nNews\nPartners\nRegister\nHome\nAbout Us\nAccreditation\nAcademic Performance\nContact\nPrincipal’s Message\nParent Liaison\nEducation Service Provider\nFAQ\nGovernance & Financials\nBoard Members\nBoard Minutes\nBoard Meetings\nBoard Speaker Form\nBudgets and Financials\nNews and Bulletin\nSchool Mission\nSomerset Academy Inc.\nTitle 1\nPartners\nPublic Records Request\nWhat are Charter Schools?\nWho We Are\nRegister\nStudents / Parents\nKey Information Bulletin\nSAC/Parent Meetings\nTutoring\nUniform Policy\nStudent Links\nClubs\nChild Abuse and Neglect Policy\nStudent Code of Conduct\nStudent Progression Plan\nSupply Lists\nSummer Reading Lists\nVolunteer Opportunities\nCalendars\nActivities Calendar\nBroward County Calendar 2017-2018\nCurriculum\nElementary Curriculum\nMiddle School Curriculum Guide\nLunch Menu\nAthletics\nAth

In [60]:
e = full_250_df[full_250_df['NCESSCH'] == id ]
e['WEBTEXT'][0]

[('https://www.somersetdade.com/', 'False', '0', '')]

255

In [178]:
len(parsed_df_7.columns)

392

In [183]:
len(parsed_df_6.columns)

394

In [182]:
len(full_250_df.columns)

392

In [185]:
len(parsed_df_1.columns)

391

In [173]:
parsed_df_7.head(1)

,AAEC,1320.0,ARIZONA AGRIBUSINESS & EQUINE CENTER - ESTRELLA,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ.1,Unnamed: 8,Unnamed: 9,...,2010.0,Unnamed: 383,"ARIZONA AGRIBUSINESS & EQUINE CENTER - ESTRELLA 3400 Dysart Rd , Avondale AZ 85007.0",239.0,0.4979166666666667,5.0.2,-0.0020833333333333333,Suburb,-480.0,0.18
0,Arizona Agribusiness & Equine Center,1320.0,ARIZONA AGRIBUSINESS & EQUINE CENTER - ESTRELLA,AZ,6.0,https://www.aaechighschools.com/,1.0,AZ,CMO,1220.0,...,2010.0,NaN,ARIZONA AGRIBUSINESS & EQUINE CENTER - ESTRELL...,239.0,0.497917,5.0,-0.002083,Suburb,-480.0,0


In [174]:
parsed_df_7.iloc[0]